In [2]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import date
from datetime import timedelta
%matplotlib inline

In [6]:
today = datetime.date.today()
yesterday = today - timedelta(days=1)

def get_sp500_tickers():
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df = table[0]
    df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
    return


def get_open_close(frame):
    cls = (web.DataReader(frame['Symbols'], data_source='yahoo', start=yesterday)['Close'])
    cls = cls.transpose().reindex()
    frame['Close'] = cls[str(yesterday)].values
    
    opn = (web.DataReader(frame['Symbols'], data_source='yahoo', start=today)['Open'])
    opn = opn.transpose().reindex()
    frame['Open'] = opn[str(today)].values
    
    return(frame)

def find_gaps(size, number_of_shares):
    today = datetime.date.today()
    yesterday = today - timedelta(days=1)
    
    print('Loading Ticker Data...')
    gap_frame = pd.read_csv("S&P500-Symbols.csv", nrows=number_of_shares)
    gap_frame.columns = ['Number', 'Symbols']
    del gap_frame['Number']
    
    print('Loading Open/Close Data...')
    gap_frame = get_open_close(gap_frame).set_index('Symbols')
    
    gap_frame['Perc Gap'] = (gap_frame['Open']-gap_frame['Close'])*100/gap_frame['Close']
    gap_frame = gap_frame[abs((gap_frame['Open']-gap_frame['Close'])*100/gap_frame['Close']) > size]
    
    return(gap_frame)

def div(x,y):
    return 0 if y==0 else x/y

def gap_analyse(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['gap size'] = (data['Open']-data['Last Close'])
    
    
    up_gap = data[data['gap size']*100 / data['Last Close'] > gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['gap size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    down_gap = data[data['gap size']*100 / data['Last Close'] < (-1)*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['return'] = (down_gap['Open']- down_gap['High'])/down_gap['gap size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=gap_perc_size, a_min=None)
    

    res = {'U Ave': [round(up_gap['return'].mean(), 2)], 'U +-': [round(up_gap['return'].std(), 2)],
           'U % Close': [round(div(len(up_gap[up_gap['return']>=1])*100,len(up_gap)))], 'No U': [len(up_gap)],
           'D Ave': [round(down_gap['return'].mean(), 2)], 'D +-': [round(down_gap['return'].std(), 2)],
           'D % Close': [round(div(len(down_gap[down_gap['return']>=1])*100, len(down_gap)), 2)], 'No D': [len(down_gap)]}
    
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: 'Average', 1: '+/-', 2:'% close', 3:'Number of Gaps'})
    
    return(results) 

get_sp500_tickers()

In [9]:
todays_gaps = find_gaps(1, 20)

Loading Ticker Data...
Loading Open/Close Data...


In [17]:
print(todays_gaps.index)

Index(['ABMD', 'AES', 'AFL', 'A', 'ALK', 'ALB', 'ALGN'], dtype='object', name='Symbols')


In [18]:
gap_results = gap_analyse(str(todays_gaps.index), '2017-05-01', 1)

RemoteDataError: No data fetched for symbol Index(['ABMD', 'AES', 'AFL', 'A', 'ALK', 'ALB', 'ALGN'], dtype='object', name='Symbols') using YahooDailyReader

In [ ]:
gap_results